In [ ]:
!pip install polars==0.18.4 lets-plot==3.2 prince==0.10.7
import numpy as np
import polars as pl
import prince
from lets_plot import *
LetsPlot.setup_html(isolated_frame=True)

# Analyse multivariée 2: ordination

Littéralement, l'ordination vise à mettre de l'*ordre* dans des données, dont le nombre élevé de variables peut amener à des difficultés d'appréciation et d'interprétaion ([Legendre et Legendre, 2012](https://www.elsevier.com/books/numerical-ecology/legendre/978-0-444-53868-0)). En science des données, le terme ordination est utilisé pour désigner un ensemble de techniques de réduction d'axe. L'analyse en composante principale est probablement la plus connue de ces techniques, mais de nombreuses techniques d'ordination ont été développées au cours des dernières années, chacune ayant ses domaines d'application.

Les techniques de réduction d'axe permettent de dégager l'information la plus importante en projetant une synthèse des relations entre les observations et entre les variables. Les techniques ne supposant aucune structure a priori sont dites *non contraignantes*. À l'inverse, les ordinations contraignantes lient des variables descriptives avec une ou plusieurs variables prédictives.

La référence en la matière est indiscutablement [Legendre et Legendre (2012)](https://www.elsevier.com/books/numerical-ecology/legendre/978-0-444-53868-0). Je m'inspirerai toutefois de l'approche du module [*Prince*](https://maxhalford.github.io/prince/), qui offre des fonctions pour l'ordination non contraignante. Dans cette section, nous ne couvrirons que l'analyse en composantes principales et l'analyse discriminante.

![](images/ord_flow-chart.png)

<small>Arbre de décision pour l'ordination, inspiré de [Max Halford](https://maxhalford.github.io/prince/)</small> 

Le module *Prince* vient avec des méthodes pour visualiser les résultats des ordinations. Jusqu'à présent, nous avons utilisé [*Lets-Plot*](https://lets-plot.org/), mais *Prince* utilise plutôt [*Altair*](https://altair-viz.github.io/) comme module graphique. Comme *Lets-Plot*, *Altair* fonctionne en mode de visualisation déclarative. Le vocabulaire est différent, mais les principes sont sensiblement les mêmes.

Plusieurs modules en Python peuvent effectuer ces opérations d'ordination.

- Analyse en composantes principales: [Prince](https://maxhalford.github.io/prince/pca/), [Scikit-Learn](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html#sklearn.decomposition.PCA) et [Statsmodels](https://www.statsmodels.org/dev/generated/statsmodels.multivariate.pca.PCA.html).
- Analyse procustéenne généralisée: [Prince](https://maxhalford.github.io/prince/gpa/).
- Analyse factorielle multiple: [Prince](https://maxhalford.github.io/prince/mfa/).
- Analyse de correspondance: [Prince](https://maxhalford.github.io/prince/ca/).
- Analyse de correspondance multiple: [Prince](https://maxhalford.github.io/prince/mfa/).
- Analyse de factorielle de données mixes: [Prince](https://maxhalford.github.io/prince/famd/).
- Échelle multidimensionnelle: [Scikit-Learn](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.MDS.html#sklearn.manifold.MDS).
- Analyse de corrélations canoniques: [Scikit-Learn](https://scikit-learn.org/stable/modules/generated/sklearn.cross_decomposition.CCA.html#sklearn.cross_decomposition.CCA)
- Analyse discriminante: [Scikit-Learn](https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html#sklearn.discriminant_analysis.LinearDiscriminantAnalysis).


## Analyse en composantes principales (ACP)

L'objectif d'une ACP est de représenter les données dans un nombre réduit de dimensions représentant le plus possible la variation d'un tableau de données : elle permet de projeter les données dans un espace dans lequel les variables sont combinées en axes orthogonaux dont le premier axe capte le maximum de variance. L'ACP peut par exemple être utilisée pour analyser des corrélations entre variables ou dégager l'information la plus pertinente d'un tableau de données météo ou de signal en un nombre plus retreint de variables.

L'ACP effectue une rotation des axes à partir de sorte que le premier axe définisse la direction à travers laquelle on retrouve la variance maximale. Ce premier axe est une combinaison linéaire des variables et forme la première composante principale. Une fois cet axe définit, on trouve le deuxième axe, orthogonal au premier, où l'on retrouve la variance maximale — cet axe forme la deuxième composante principale, et ainsi de suite jusqu'à ce que le nombre d'axes corresponde au nombre de variables.

Les projections des observations sur ces axes principaux sont appelées les **scores**. Les projections des variables sur les axes principaux sont les **vecteurs propres**  (*eigenvectors*, ou *loadings*). La variance des composantes principales diminue de la première à la dernière, et peut être calculée comme une proportion de la variance totale : c'est le **pourcentage d'inertie**. Par convention, on utilise les **valeurs propres** (*eigenvalues*) pour mesurer l'importance des axes. Si la première composante principale a une inertie de 50% et la deuxième a une inertie de 30%, la représentation en 2D des projections représentera 50% + 30% = 80% de la variance du nuage de points.

L'hétérogénéité des échelles de mesure peut avoir une grande importance sur les résultats d'une ACP (les données doivent être dimensionnellement homogènes). En effet, la hauteur d'un cerisier aura une variance plus grande que le diamètre d'une cerise exprimé dans les mêmes unités, et cette dernière aura plus de variance que la teneur en cuivre d'une feuille. Il est conséquemment avisé d'homogénéiser les échelles en centrant la moyenne à zéro et l'écart-type à 1 avant de procéder à une ACP.

L'ACP a été conçue pour projeter en un nombre moindre de dimensions des observations dont les distributions sont multinormales. Bien que l'ACP soit une technique robuste, il est préférable de transformer préalablement les variables dont la distribution est particulièrement asymétrique ([Legendre et Legendre, 2012, p. 450](https://www.elsevier.com/books/numerical-ecology/legendre/978-0-444-53868-0)). Le cas échéant, les valeurs extrêmes pourraient faire dévier les vecteurs propres et biaiser l'analyse. En particulier, les données ACP menées sur des données compositionnelles sont réputées pour générer des analyses biaisées ([Pawlowsky-Glahn and Egozcue, 2006](http://sp.lyellcollection.org/content/specpubgsl/264/1/1.full.pdf)). Le test de Mardia ([Korkmaz, 2014](https://journal.r-project.org/archive/2014-2/korkmaz-goksuluk-zararsiz.pdf)) peut être utilisé pour tester la multinormalité. Une distribution multinormale devrait générer des scores en forme d'hypersphère (en forme de cercle sur un biplot: voir plus loin).

Voyons ce que nous dira une ACP sur nos données de manchots.

In [ ]:
penguins = pl.read_csv('data/penguins.csv', null_values='NA')
penguins = (
    penguins
    .filter(pl.all(pl.col(pl.Float32, pl.Float64).is_not_nan())) # enlever les NaN
    .drop_nulls() # enlever les None
)

J'ai pris la liberté de nettoyer le tableau `penguins` en enlevant les valeurs NaN (*not a number*) et les valeurs *None*. L'ACP est généralement effectuée sur les *features* (variables prédictives). J'inclus le nom des *features* de mon tableau dans une liste pour la convivialité. Je prends cette liste pour créer un nouveau tableau. Puisque le module *Prince* n'accepte que les tableaux de type *Pandas*, j'effectue la conversion. Prenez note qu'un tableau *Pandas* ne possède pas les mêmes propriétés qu'un tableau *Polars*.

In [ ]:
ordination_features = ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']
penguins_ord_df = penguins.select(ordination_features).to_pandas()

*Prince* fonctionne comme *Scikit-Learn* (élaboré dans la prochaine section): instanciation de l'objet (l'objet étant le modèle), puis lissage sur les données.

In [ ]:
# instanciation de l'objet
penguins_pca = prince.PCA(
    n_components=len(ordination_features),
    n_iter=10,
    rescale_with_mean=True,
    rescale_with_std=True,
    copy=True,
    check_input=True,
    engine='sklearn'
)

#lissage
## to_pandas parce que Prince ne prend pas les tableaux Polars, mais Pandas
penguins_pca.fit(penguins_ord_df) 

L'objet lissé (ma traduction de *fitted*) contient toutes les informations nécessaires pour interpréter l'ACP. Le sommaire des valeurs propres permet de connaître le pourcentage cumulatif de la variance captée par les composantes, un indicateur important pour interpréter un biplot (voir plus loin) ou pour décider si l'information des variables peut être condensée en un nombre moindre de variables composites. On peut aller chercher les valeurs propres dans le sommaire par `penguins_pca.eigenvalues_summary`. Ou bien, en lançant la visualisation des valeurs propres. Le graphique étant interactif, vous verrez davantage d'informations et plaçant le curseur sur les barres.

In [ ]:
penguins_pca.scree_plot()

L'objet `penguins_pca` contient aussi les informations brutes:

- valeurs propres
- les scores, qui représentent les observations
- loadings, qui représentent les variables 

In [ ]:
# les valeurs propres
penguins_ev = penguins_pca.eigenvalues_ 

# les scores
penguins_scores = penguins_pca.transform(penguins_ord_df)

# les loadings
penguins_loadings = penguins_pca.column_coordinates_

### Représentation en biplot

Les scores et les loadings permettent de représenter les données sous forme de biplot. Mais, préalablement, [Borcard et al. 2011](https://link.springer.com/article/10.1007/s13253-012-0094-x) proposent d'effectuer consciencieusement une mise à l'échelle (*scaling*). En gros, vous préférerez le **scaling 1** pour un biplot de distance, où la distance entre les objets (observations et variables) dans le biplot est proche des distances entre les objets dans les données de haute dimension. Vous préférerez le **scaling 2** pour les biplots de corrélation, où les angles entre les chargements (avec des segments reliant les chargements à l'origine) sont proches de leur corrélation (lisez [Borcard et al. 2011](https://link.springer.com/article/10.1007/s13253-012-0094-x) pour plus de détails). À ma connaissance, les autres échelles sont rarement utilisées. De plus, bien que peu d'articles scientifiques se soucient de l'échelle, elle affecte considérablement l'interprétation: mieux vaut spécifier le scaling lorsque vous présentez un biplot.


![](images/pca_scaling.png)

Capture d'écran de https://rdrr.io/rforge/vegan/f/inst/doc/decision-vegan.pdf

La fonction suivante effectue le scaling.

In [ ]:
def eigen_scaling(scores, loadings, eigenvalues, scaling = 0):
    """
    pca is a PCA object obtained from statsmodels.multivariate.pca
    scaling is one of [0, 1, 2, 3]
    the eigenvalues of the pca object are n times the ones computed with R
    we thus need to divide their sum by the number of rows
    """
   
    const = ((scores.shape[0]-1) * eigenvalues.sum()/ scores.shape[0])**0.25
    if scaling == 0:
        scores = scores
        loadings = loadings
    elif scaling == 1:
        scaling_fac = (eigenvalues / eigenvalues.sum())**0.5
        scores = scores * scaling_fac * const
        loadings = loadings * const
    elif scaling == 2:
        scaling_fac = (eigenvalues / eigenvalues.sum())**0.5
        scores = scores * const
        loadings = loadings * scaling_fac * const
    elif scaling == 3:
        scaling_fac = (eigenvalues / eigenvalues.sum())**0.25
        scores = scores * scaling_fac * const
        loadings = loadings * scaling_fac * const
    else:
        sys.exit("Scaling should either be 0, 1, 2 or 3")
    return([scores, loadings])


J'effectue un `scaling = 2` pour avoir une idée des corrélations entre les variables.

In [ ]:
penguins_scores_sc, penguins_loadings_sc = eigen_scaling(
    scores = penguins_scores,
    loadings = penguins_loadings,
    eigenvalues = penguins_ev,
    scaling = 2
)

Les entêtes des tableaux des scores et des loadings sont des valeurs numériques, ce qui complique leur utilisation subséquente pour les graphiques avec *Lets-Plot*. Les manipulations de la prochaine cellule consistent à transformer les tableaux de sortie de *Prince* en format *Polars*, à changer le nom des colonnes par PC0, PC1, PC2 et PC3. Pour les scores, j'ajoute une colonne identifiant l'espèce. Enfin, puis les loadings, j'ajoute une colonne identifiant l'attribut (*feature*).

In [ ]:
pca_colnames = ['PC' + str(i) for i in range(len(ordination_features))]
penguins_scores_sc = (
    pl.DataFrame(penguins_scores_sc, schema = pca_colnames)
    .with_columns(penguins.select('species'))
)
penguins_loadings_sc = (
    pl.DataFrame(penguins_loadings_sc, schema = pca_colnames)
    .with_columns(pl.Series(ordination_features).alias('feature'))
)

Nos tableaux nous permettent de créer un biplot avec *Lets-Plot*.

In [ ]:
(
    ggplot()
    + geom_point(
        data = penguins_scores_sc,
        mapping = aes(x='PC0', y='PC1', color='species')
    )
    + geom_segment(
        data = penguins_loadings_sc,
        mapping = aes(xend = 'PC0', yend = 'PC1'),
        x = 0, y = 0, color = 'black'
    )
    + geom_text(
        data = penguins_loadings_sc,
        mapping = aes(x = 'PC0', y = 'PC1', label = 'feature')
    )
)

La représentation en biplot est ce qu'on retrouverait si on écrasait en 2D les données selon la perspective où les données sont le plus éclatées. Voici quelques interprétations du biplot:

- Étant donnée qu'on est en scaling 2, nous pouvons conclure que la longueur des ailes et la masse du manchot sont positivement corrélés. Ces deux attributs sont positivement corrélés, mais dans une moindre mesure, avec la longueur du bec, mais négativement corrélés avec l'épaisseur du bec.
- Les Gentoos sont différents des Adelie et Chinstrap, qui se ressemblent davantage. Les Gentoos se différencient des autres par leur masse plus élevée, leurs ailes plus longues, et leur bec plus long. Les Adelies et des Chinstraps se recoupent passablement, mais peuvent se différencier selon la longueur de leur bec.

![](images/Penguin-heights.jpg)

Source: https://www.bas.ac.uk/about/antarctica/wildlife/penguins/

## Analyse discriminante (DA)

L'analyse discriminante est une technique d'ordination utilisée pour distinguer ou prédire des catégories. Elle cherche à trouver les combinaisons de variables qui permettent de séparer au mieux les différentes catégories.

Nous allons utiliser la classe `LinearDiscriminantAnalysis` de `scikit-learn` pour effectuer une analyse discriminante sur un ensemble de données.

Nous devons d'abord convertir la colonne `species` en valeurs numériques.

In [ ]:
penguins = (
    penguins
    .with_columns(pl.col('species').cast(pl.Categorical).cast(pl.UInt32).alias('species_int'))
)

À ce stade, je vous laisse interpréter mon code!

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
penguins_sc = scaler.fit_transform(penguins.select(ordination_features))

# Analyse discriminante
penguins_lda = LDA(n_components=2)
penguins_lda.fit(
    penguins_sc,
    penguins.select('species_int')
)

# L'extraction des attributs est différente qu'avec Prince
lda_scores = penguins_lda.transform(penguins_sc)
lda_loadings = penguins_lda.coef_.T[:, :lda_scores.shape[1]]
lda_eigenvalues = penguins_lda.explained_variance_ratio_

Notez que `penguins_lda.explained_variance_ratio_` extrait un ratio, qu'il faudrait multiplier par la variance `penguins.select(ordination_features).std()**2`, mais celle-ci est unitaire étant donné que l'on a préalablement mis les données à l'échelle avec `StandardScaler()`.

In [ ]:
# Scaling
penguins_lda_scores, penguins_lda_loadings = eigen_scaling(
    scores = lda_scores,
    loadings = lda_loadings,
    eigenvalues = lda_eigenvalues,
    scaling = 2
)

# Transformer en format Polars, et renommer les variables
lda_colnames = ['DA0', 'DA1']
penguins_lda_scores = (
    pl.DataFrame(penguins_lda_scores, schema = lda_colnames)
    .with_columns(penguins.select('species'))
)
penguins_lda_loadings = (
    pl.DataFrame(penguins_lda_loadings)
    .with_columns(pl.Series(ordination_features).alias('features'))
)

Et le biplot en scaling 2!

In [ ]:
(
    ggplot(data=penguins_lda_scores)
    + geom_point(aes(x='DA0', y='DA1', color='species'))
    + geom_segment(
        data = penguins_lda_loadings,
        mapping = aes(xend = 'column_0', yend='column_1'),
        x = 0, y = 0, color = 'black'
    )
    + geom_text(
        data = penguins_lda_loadings,
        mapping = aes(x = 'column_0', y = 'column_1', label='features')
    )
)

À la différence d'une analyse en composantes principales, l'analyse discriminante écrase en 2D la perspective où les groupes sont les plus éclatés. Pouvez-vous interpréter le biplot?